importing libraries

In [1]:
import pandas as pd
import numpy as np
from openai import OpenAI
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score

/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Reduced Dataset is the portion of the initial Dataset that was complete

In [2]:
reduced_df = pd.read_csv('/home/profniggastein/PycharmProjects/ReducedDataset/Project_CodeNet/Reduced_df.csv')
reduced_df

/tmp/ipykernel_2086/1069376804.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  reduced_df = pd.read_csv('/home/profniggastein/PycharmProjects/ReducedDataset/Project_CodeNet/Reduced_df.csv')


,Unnamed: 0,index,submission_id,problem_id,user_id,date,language,original_language,filename_ext,status,...,memory,code_size,accuracy,Correct/incorrect,submission_idstr,problem_idstr,test_path,sum_path,code,summary
0,4850,8415,s667847559,p00000,u972675635,1530881789,C++,C++11,cpp,Accepted,...,3108.0,250,1/1,1,s667847559,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...
1,4851,8416,s160425098,p00000,u642752018,1530897493,C++,C++,cpp,Accepted,...,3080.0,232,1/1,1,s160425098,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...
2,4852,8417,s682094990,p00000,u313584000,1531051141,C++,C++,cpp,Accepted,...,3096.0,182,1/1,1,s682094990,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...
3,4854,8419,s068595017,p00000,u011621222,1531399592,C++,C++,cpp,Accepted,...,2588.0,129,1/1,1,s068595017,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<cstdio>\nint main(){\n\tfor(int i = 1...,Here's the code summary for the given problem ...
4,4857,8422,s224013242,p00000,u937856796,1531468490,C++,C++,cpp,Accepted,...,3080.0,180,1/1,1,s224013242,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\n\ni...,Here's the code summary for the given problem ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,6818156,2073,s921588169,p04050,u748871552,1470008541,C++,C++11 (GCC 4.8.1),cpp,Accepted,...,256.0,1080,NaN,1,s921588169,p04050,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,"#include <bits/stdc++.h>\n\n#define REP(i,n) f...",Here is the code summary format for the given ...
499996,6818157,2074,s191865152,p04050,u955723966,1469987618,C++,C++14 (GCC 5.4.1),cpp,Accepted,...,256.0,1066,NaN,1,s191865152,p04050,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <bits/stdc++.h>\n\nusing namespace st...,Here is the code summary format for the given ...
499997,6818166,2083,s564884250,p04050,u572012381,1469778717,C++,C++14 (GCC 5.4.1),cpp,Accepted,...,256.0,1577,NaN,1,s564884250,p04050,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <bits/stdc++.h>\nusing namespace std;...,Here is the code summary format for the given ...
499998,6818168,2085,s162996393,p04050,u657011188,1469479168,C++,C++14 (GCC 5.4.1),cpp,Accepted,...,256.0,1021,NaN,1,s162996393,p04050,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <bits/stdc++.h>\nusing namespace std;...,Here is the code summary format for the given ...


Using a small sample of the first 100 entries as a test for the summarisation pipeline

In [3]:
df = reduced_df.copy()
df_sample = df.head(10).reset_index(drop=True)
#delete the original df variable to save memory
del df


In [4]:
df = df_sample
df

,Unnamed: 0,index,submission_id,problem_id,user_id,date,language,original_language,filename_ext,status,...,memory,code_size,accuracy,Correct/incorrect,submission_idstr,problem_idstr,test_path,sum_path,code,summary
0,4850,8415,s667847559,p00000,u972675635,1530881789,C++,C++11,cpp,Accepted,...,3108.0,250,1/1,1,s667847559,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...
1,4851,8416,s160425098,p00000,u642752018,1530897493,C++,C++,cpp,Accepted,...,3080.0,232,1/1,1,s160425098,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...
2,4852,8417,s682094990,p00000,u313584000,1531051141,C++,C++,cpp,Accepted,...,3096.0,182,1/1,1,s682094990,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...
3,4854,8419,s068595017,p00000,u011621222,1531399592,C++,C++,cpp,Accepted,...,2588.0,129,1/1,1,s068595017,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<cstdio>\nint main(){\n\tfor(int i = 1...,Here's the code summary for the given problem ...
4,4857,8422,s224013242,p00000,u937856796,1531468490,C++,C++,cpp,Accepted,...,3080.0,180,1/1,1,s224013242,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\n\ni...,Here's the code summary for the given problem ...
5,4858,8423,s864308147,p00000,u011621222,1531653690,C++,C++11,cpp,Accepted,...,2500.0,217,1/1,1,s864308147,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<stdio.h>\n\nint main()\n{\n\n int ...,Here's the code summary for the given problem ...
6,4859,8424,s558211984,p00000,u881776454,1531760031,C++,C++,cpp,Accepted,...,3076.0,815,1/1,1,s558211984,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <set>\n#include <map>\n#include <list...,Here's the code summary for the given problem ...
7,4860,8425,s342884441,p00000,u401240263,1531916188,C++,C++,cpp,Accepted,...,3120.0,171,1/1,1,s342884441,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <bits/stdc++.h>\nusing namespace std;...,Here's the code summary for the given problem ...
8,4861,8426,s912845136,p00000,u098170735,1531931333,C++,C++,cpp,Accepted,...,3048.0,183,1/1,1,s912845136,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...
9,4862,8427,s213469359,p00000,u027430658,1532008267,C++,C++,cpp,Accepted,...,3000.0,244,1/1,1,s213469359,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\n#include <iomanip>\n#defi...,Here's the code summary for the given problem ...


Using the DeepSeek model to generate summaries for the code submissions in the dataset.              In this test the base deepseek model is used and gives satisfactoy results at least for the sample data ,in proper implementation the plan is to use the deepsek reasoning model for better results.

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model_id = "deepseek-ai/deepseek-coder-6.7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16 if device == "cuda" else torch.float32)
model.to(device)


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [02:12<00:00, 66.16s/it]
/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaLinearScalingRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )

In [6]:
def GenerateSummary(text):
    prompt = f"""<|system|>
You are are an assistant/tutor helping users understand various code snippets by giving a general summary and as much as possible provide it in a general summary form for specifically C and C++ code like this:

Code Summary Format

1. Function/Method Name:
process_data

2. Purpose:
Briefly describe what the code is intended to do.
E.g., Processes and cleans input data by removing null values and standardizing formats.

3. Inputs:
dataframe (pandas.DataFrame): The input dataset containing raw values.
columns (List[str]): Specific columns to be processed.

4. Outputs:
cleaned_dataframe (pandas.DataFrame): A cleaned version of the input dataset.

5. Key Operations/Logic:
- Drops rows with null values in specified columns
- Converts column names to lowercase
- Normalizes date formats

6. Dependencies:
- pandas
- datetime

7. Edge Cases Handled:
- Returns an empty DataFrame if all rows contain nulls
- Skips processing if columns is empty

8. Example Usage (Optional):
clean_df = process_data(df, ["date", "price", "location"])
<|user|>
{text}
<|assistant|>"""

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(**inputs, max_new_tokens=512)
    return tokenizer.decode(output[0], skip_special_tokens=True).split("<|assistant|>")[-1].strip()


In [7]:
# client = OpenAI(api_key="sk-9a18103e17be4329a06b65b1520c5d9b", base_url="https://api.deepseek.com")
#
# def GenerateSummary(text):
#
#     response = client.chat.completions.create(
#         model="deepseek-chat",
#         messages=[
#             {"role": "system", "content": """You are are an assistant/tutor helping users understand various code snippets by giving a general summary and as much as poosible give them  in a general summary form for specifically c and c++ code like this;Code Summary Format
#
#     1. Function/Method Name:
#     process_data
#
#     2. Purpose:
#     Briefly describe what the code is intended to do.
#     E.g., Processes and cleans input data by removing null values and standardizing formats.
#
#     3. Inputs:
#
#     dataframe (pandas.DataFrame): The input dataset containing raw values.
#
#     columns (List[str]): Specific columns to be processed.
#
#
#     4. Outputs:
#
#     cleaned_dataframe (pandas.DataFrame): A cleaned version of the input dataset.
#
#
#     5. Key Operations/Logic:
#
#     Drops rows with null values in specified columns
#
#     Converts column names to lowercase
#
#     Normalizes date formats
#
#
#     6. Dependencies:
#
#     pandas
#
#     datetime
#
#
#     7. Edge Cases Handled:
#
#     Returns an empty DataFrame if all rows contain nulls
#
#     Skips processing if columns is empty
#
#
#     8. Example Usage (Optional):
#
#     clean_df = process_data(df, ["date", "price", "location"])"""},
#             {"role": "user", "content": f"{text}"},
#         ],
#         stream=False
#     )
#     return response
#


In [8]:
df = df_sample
df['GenSummary'] = df['code'].apply(lambda x: GenerateSummary(x) if pd.notnull(x) else None)

df

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


,Unnamed: 0,index,submission_id,problem_id,user_id,date,language,original_language,filename_ext,status,...,code_size,accuracy,Correct/incorrect,submission_idstr,problem_idstr,test_path,sum_path,code,summary,GenSummary
0,4850,8415,s667847559,p00000,u972675635,1530881789,C++,C++11,cpp,Accepted,...,250,1/1,1,s667847559,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Function/Method Name:\nmain\n\nPurpose:\nThis ...
1,4851,8416,s160425098,p00000,u642752018,1530897493,C++,C++,cpp,Accepted,...,232,1/1,1,s160425098,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Function/Method Name:\nmultiplication_table_cu...
2,4852,8417,s682094990,p00000,u313584000,1531051141,C++,C++,cpp,Accepted,...,182,1/1,1,s682094990,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...,"Sure, I'd be happy to explain that.\n\nIn C++,..."
3,4854,8419,s068595017,p00000,u011621222,1531399592,C++,C++,cpp,Accepted,...,129,1/1,1,s068595017,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<cstdio>\nint main(){\n\tfor(int i = 1...,Here's the code summary for the given problem ...,Code Summary Format\n\n1. Function/Method Name...
4,4857,8422,s224013242,p00000,u937856796,1531468490,C++,C++,cpp,Accepted,...,180,1/1,1,s224013242,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\n\ni...,Here's the code summary for the given problem ...,Code Summary Format\n\n1. Function/Method Name...
5,4858,8423,s864308147,p00000,u011621222,1531653690,C++,C++11,cpp,Accepted,...,217,1/1,1,s864308147,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<stdio.h>\n\nint main()\n{\n\n int ...,Here's the code summary for the given problem ...,
6,4859,8424,s558211984,p00000,u881776454,1531760031,C++,C++,cpp,Accepted,...,815,1/1,1,s558211984,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <set>\n#include <map>\n#include <list...,Here's the code summary for the given problem ...,Code Summary Format\n\n1. Function/Method Name...
7,4860,8425,s342884441,p00000,u401240263,1531916188,C++,C++,cpp,Accepted,...,171,1/1,1,s342884441,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <bits/stdc++.h>\nusing namespace std;...,Here's the code summary for the given problem ...,Code Summary Format\n\n1. Function/Method Name...
8,4861,8426,s912845136,p00000,u098170735,1531931333,C++,C++,cpp,Accepted,...,183,1/1,1,s912845136,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Function/Method Name:\nmultiplication_table_cu...
9,4862,8427,s213469359,p00000,u027430658,1532008267,C++,C++,cpp,Accepted,...,244,1/1,1,s213469359,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\n#include <iomanip>\n#defi...,Here's the code summary for the given problem ...,"Sure, you can run this code in any C++ compile..."


In [9]:
df.to_csv('/home/profniggastein/PycharmProjects/ReducedDataset/Project_CodeNet/Reduced_localsample_df.csv')
df

#df_sample

,Unnamed: 0,index,submission_id,problem_id,user_id,date,language,original_language,filename_ext,status,...,code_size,accuracy,Correct/incorrect,submission_idstr,problem_idstr,test_path,sum_path,code,summary,GenSummary
0,4850,8415,s667847559,p00000,u972675635,1530881789,C++,C++11,cpp,Accepted,...,250,1/1,1,s667847559,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Function/Method Name:\nmain\n\nPurpose:\nThis ...
1,4851,8416,s160425098,p00000,u642752018,1530897493,C++,C++,cpp,Accepted,...,232,1/1,1,s160425098,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Function/Method Name:\nmultiplication_table_cu...
2,4852,8417,s682094990,p00000,u313584000,1531051141,C++,C++,cpp,Accepted,...,182,1/1,1,s682094990,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...,"Sure, I'd be happy to explain that.\n\nIn C++,..."
3,4854,8419,s068595017,p00000,u011621222,1531399592,C++,C++,cpp,Accepted,...,129,1/1,1,s068595017,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<cstdio>\nint main(){\n\tfor(int i = 1...,Here's the code summary for the given problem ...,Code Summary Format\n\n1. Function/Method Name...
4,4857,8422,s224013242,p00000,u937856796,1531468490,C++,C++,cpp,Accepted,...,180,1/1,1,s224013242,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\n\ni...,Here's the code summary for the given problem ...,Code Summary Format\n\n1. Function/Method Name...
5,4858,8423,s864308147,p00000,u011621222,1531653690,C++,C++11,cpp,Accepted,...,217,1/1,1,s864308147,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<stdio.h>\n\nint main()\n{\n\n int ...,Here's the code summary for the given problem ...,
6,4859,8424,s558211984,p00000,u881776454,1531760031,C++,C++,cpp,Accepted,...,815,1/1,1,s558211984,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <set>\n#include <map>\n#include <list...,Here's the code summary for the given problem ...,Code Summary Format\n\n1. Function/Method Name...
7,4860,8425,s342884441,p00000,u401240263,1531916188,C++,C++,cpp,Accepted,...,171,1/1,1,s342884441,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <bits/stdc++.h>\nusing namespace std;...,Here's the code summary for the given problem ...,Code Summary Format\n\n1. Function/Method Name...
8,4861,8426,s912845136,p00000,u098170735,1531931333,C++,C++,cpp,Accepted,...,183,1/1,1,s912845136,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Function/Method Name:\nmultiplication_table_cu...
9,4862,8427,s213469359,p00000,u027430658,1532008267,C++,C++,cpp,Accepted,...,244,1/1,1,s213469359,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\n#include <iomanip>\n#defi...,Here's the code summary for the given problem ...,"Sure, you can run this code in any C++ compile..."


for evaluating the generated sumaries BLEU and BERT score are the metrics used to compare the generated summaries with the original summaries in the dataset. the BERT score is calculated using the BERTScore library instead of a "manual" Bert model implimentation while this implimentation is faster access to the acc embedding-space is lost

In [11]:


def calculate_bleu(reference, candidate):
    reference = reference.split()
    candidate = candidate.split()
    return sentence_bleu([reference], candidate, smoothing_function=SmoothingFunction().method1)
df['bleu_score'] = df.apply(
    lambda row: calculate_bleu(row['GenSummary'], row['summary']) if pd.notnull(row['summary']) and pd.notnull(row['GenSummary']) else None,
    axis=1
)
df['BERT_score'] = df.apply(
    lambda row: score([row["GenSummary"]], [row["summary"]], model_type="roberta-large")[2][0].item(),
    axis=1
)
df


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta

,Unnamed: 0,index,submission_id,problem_id,user_id,date,language,original_language,filename_ext,status,...,Correct/incorrect,submission_idstr,problem_idstr,test_path,sum_path,code,summary,GenSummary,bleu_score,BERT_score
0,4850,8415,s667847559,p00000,u972675635,1530881789,C++,C++11,cpp,Accepted,...,1,s667847559,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Function/Method Name:\nmain\n\nPurpose:\nThis ...,0.003794,0.802451
1,4851,8416,s160425098,p00000,u642752018,1530897493,C++,C++,cpp,Accepted,...,1,s160425098,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Function/Method Name:\nmultiplication_table_cu...,0.023057,0.835466
2,4852,8417,s682094990,p00000,u313584000,1531051141,C++,C++,cpp,Accepted,...,1,s682094990,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...,"Sure, I'd be happy to explain that.\n\nIn C++,...",0.005445,0.803933
3,4854,8419,s068595017,p00000,u011621222,1531399592,C++,C++,cpp,Accepted,...,1,s068595017,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<cstdio>\nint main(){\n\tfor(int i = 1...,Here's the code summary for the given problem ...,Code Summary Format\n\n1. Function/Method Name...,0.004261,0.810047
4,4857,8422,s224013242,p00000,u937856796,1531468490,C++,C++,cpp,Accepted,...,1,s224013242,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\n\ni...,Here's the code summary for the given problem ...,Code Summary Format\n\n1. Function/Method Name...,0.009122,0.816862
5,4858,8423,s864308147,p00000,u011621222,1531653690,C++,C++11,cpp,Accepted,...,1,s864308147,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<stdio.h>\n\nint main()\n{\n\n int ...,Here's the code summary for the given problem ...,,0.000000,0.000000
6,4859,8424,s558211984,p00000,u881776454,1531760031,C++,C++,cpp,Accepted,...,1,s558211984,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <set>\n#include <map>\n#include <list...,Here's the code summary for the given problem ...,Code Summary Format\n\n1. Function/Method Name...,0.001393,0.796439
7,4860,8425,s342884441,p00000,u401240263,1531916188,C++,C++,cpp,Accepted,...,1,s342884441,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <bits/stdc++.h>\nusing namespace std;...,Here's the code summary for the given problem ...,Code Summary Format\n\n1. Function/Method Name...,0.052827,0.841073
8,4861,8426,s912845136,p00000,u098170735,1531931333,C++,C++,cpp,Accepted,...,1,s912845136,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Function/Method Name:\nmultiplication_table_cu...,0.029826,0.839478
9,4862,8427,s213469359,p00000,u027430658,1532008267,C++,C++,cpp,Accepted,...,1,s213469359,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\n#include <iomanip>\n#defi...,Here's the code summary for the given problem ...,"Sure, you can run this code in any C++ compile...",0.015018,0.832799


In [33]:
df.to_csv('/home/profniggastein/PycharmProjects/ReducedDataset/Project_CodeNet/Reduced_localsample_df_Evaluated.csv', index=False)
df

,Unnamed: 0,index,submission_id,problem_id,user_id,date,language,original_language,filename_ext,status,...,Correct/incorrect,submission_idstr,problem_idstr,test_path,sum_path,code,summary,GenSummary,bleu_score,BERT_score
0,4850,8415,s667847559,p00000,u972675635,1530881789,C++,C++11,cpp,Accepted,...,1,s667847559,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Here's the summary of the provided C++ code:\n...,0.185897,0.777436
1,4851,8416,s160425098,p00000,u642752018,1530897493,C++,C++,cpp,Accepted,...,1,s160425098,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,### Code Summary Format \n\n1. **Function/Met...,0.034863,0.728369
2,4852,8417,s682094990,p00000,u313584000,1531051141,C++,C++,cpp,Accepted,...,1,s682094990,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...,Here’s a structured summary of the provided C+...,0.160364,0.787236
3,4854,8419,s068595017,p00000,u011621222,1531399592,C++,C++,cpp,Accepted,...,1,s068595017,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<cstdio>\nint main(){\n\tfor(int i = 1...,Here's the code summary for the given problem ...,Here’s a structured summary of the provided C+...,0.051598,0.747192
4,4857,8422,s224013242,p00000,u937856796,1531468490,C++,C++,cpp,Accepted,...,1,s224013242,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\n\ni...,Here's the code summary for the given problem ...,Here’s the structured summary of the provided ...,0.162871,0.796201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5407,8972,s729213997,p00000,u910106747,1424452541,C++,C++,cpp,Accepted,...,1,s729213997,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,"\n#include ""iostream""\n\nusing namespace std;\...",Here's the code summary for the given problem ...,Here’s a structured summary of the provided C+...,0.074141,0.741793
96,5415,8980,s317924322,p00000,u460034312,1424877773,C++,C++,cpp,Accepted,...,1,s317924322,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Here’s the structured summary of the provided ...,0.225518,0.815287
97,5416,8981,s615960680,p00000,u014398719,1425114218,C++,C++,cpp,Accepted,...,1,s615960680,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\n\nusing namespace std;\n\...,Here's the code summary for the given problem ...,Here’s the structured summary of the provided ...,0.167847,0.789103
98,5538,9103,s871011653,p00000,u080308822,1425519637,C++,C++,cpp,Accepted,...,1,s871011653,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...,Here’s a structured summary of the provided C+...,0.079642,0.747774
